<a href="https://colab.research.google.com/github/carolinampessoa/TechChallengeFase1/blob/main/TechChallengeFase5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalação de libs

!pip install openai

In [ ]:
#Configurar API Key (uso de LLM da OpenAI)

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Digite sua OpenAI API Key: ")

Digite sua OpenAI API Key: ··········


In [ ]:
#Importar imagem para avaliação

from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]

print("Imagem carregada:", image_path)


Saving Screenshot_1.png to Screenshot_1 (2).png
Imagem carregada: Screenshot_1 (2).png


In [ ]:
# Funções auxiliares

from openai import OpenAI
import base64
import json

client = OpenAI()

def encode_image(path):
    with open(path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

base64_image = encode_image(image_path)

In [ ]:
# Extração Livre de Componentes

free_extraction_prompt = """
Analise o diagrama de arquitetura de software presente na imagem.

Identifique TODOS os elementos arquiteturais semanticamente relevantes.

Não restrinja a categorias pré-definidas.

Responda exclusivamente em JSON válido:

{
  "components": [
    {"name": "...", "type": "..."}
  ]
}
"""

response1 = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": free_extraction_prompt},
                {
                    "type": "input_image",
                    "image_url": f"data:image/png;base64,{base64_image}"
                }
            ]
        }
    ]
)

raw_components_text = response1.output_text
print(raw_components_text)


```json
{
  "components": [
    {"name": "Usuários SEI", "type": "Usuários"},
    {"name": "AWS Shield", "type": "Serviço de Proteção contra DDoS"},
    {"name": "Amazon CloudFront", "type": "Distribuição de Conteúdo CDN"},
    {"name": "AWS WAF", "type": "Firewall de Aplicação Web"},
    {"name": "AWS Cloud", "type": "Nuvem Pública"},
    {"name": "sa-east-1 (São Paulo)", "type": "Região AWS"},
    {"name": "Virtual Private Cloud", "type": "Rede Virtual Privada"},
    {"name": "Availability Zone A", "type": "Zona de Disponibilidade"},
    {"name": "Availability Zone B", "type": "Zona de Disponibilidade"},
    {"name": "Availability Zone C", "type": "Zona de Disponibilidade"},
    {"name": "Public Subnet", "type": "Subnet Pública"},
    {"name": "Private Subnet", "type": "Subnet Privada"},
    {"name": "Application Load Balancer", "type": "Balanceador de Carga de Aplicação"},
    {"name": "SEI / SIP (Auto Scaling API Server)", "type": "Serviço de API com Auto Scaling"},
    {"name": "A

In [ ]:
# Limpeza + Parsing

clean_text = raw_components_text.strip()

if clean_text.startswith("```"):
    clean_text = clean_text.replace("```json", "").replace("```", "").strip()

raw_components = json.loads(clean_text)["components"]

raw_components


[{'name': 'Usuários SEI', 'type': 'Usuários'},
 {'name': 'AWS Shield', 'type': 'Serviço de Proteção contra DDoS'},
 {'name': 'Amazon CloudFront', 'type': 'Distribuição de Conteúdo CDN'},
 {'name': 'AWS WAF', 'type': 'Firewall de Aplicação Web'},
 {'name': 'AWS Cloud', 'type': 'Nuvem Pública'},
 {'name': 'sa-east-1 (São Paulo)', 'type': 'Região AWS'},
 {'name': 'Virtual Private Cloud', 'type': 'Rede Virtual Privada'},
 {'name': 'Availability Zone A', 'type': 'Zona de Disponibilidade'},
 {'name': 'Availability Zone B', 'type': 'Zona de Disponibilidade'},
 {'name': 'Availability Zone C', 'type': 'Zona de Disponibilidade'},
 {'name': 'Public Subnet', 'type': 'Subnet Pública'},
 {'name': 'Private Subnet', 'type': 'Subnet Privada'},
 {'name': 'Application Load Balancer',
  'type': 'Balanceador de Carga de Aplicação'},
 {'name': 'SEI / SIP (Auto Scaling API Server)',
  'type': 'Serviço de API com Auto Scaling'},
 {'name': 'Amazon Elastic File System (NFS) Multi-AZ',
  'type': 'Sistema de Arqu

In [ ]:
# Normalização Taxonômica

normalization_prompt = f"""
Normalize os componentes abaixo em categorias de segurança.

Categorias permitidas (escolha apenas UMA por item):

- user
- server
- database
- api
- external_system

Componentes:

{json.dumps(raw_components, indent=2)}

Responda em JSON válido:

{{
  "components": [
    {{"name": "...", "type": "..."}}
  ]
}}
"""

response2 = client.responses.create(
    model="gpt-4.1-mini",
    input=normalization_prompt
)

normalized_text = response2.output_text
print(normalized_text)


```json
{
  "components": [
    {"name": "Usuários SEI", "type": "user"},
    {"name": "AWS Shield", "type": "external_system"},
    {"name": "Amazon CloudFront", "type": "external_system"},
    {"name": "AWS WAF", "type": "external_system"},
    {"name": "AWS Cloud", "type": "external_system"},
    {"name": "sa-east-1 (São Paulo)", "type": "external_system"},
    {"name": "Virtual Private Cloud", "type": "server"},
    {"name": "Availability Zone A", "type": "external_system"},
    {"name": "Availability Zone B", "type": "external_system"},
    {"name": "Availability Zone C", "type": "external_system"},
    {"name": "Public Subnet", "type": "server"},
    {"name": "Private Subnet", "type": "server"},
    {"name": "Application Load Balancer", "type": "server"},
    {"name": "SEI / SIP (Auto Scaling API Server)", "type": "api"},
    {"name": "Amazon Elastic File System (NFS) Multi-AZ", "type": "server"},
    {"name": "Amazon RDS (Primary)", "type": "database"},
    {"name": "Amazon RDS 

In [ ]:
# Parsing Normalizado

clean_text = normalized_text.strip()

if clean_text.startswith("```"):
    clean_text = clean_text.replace("```json", "").replace("```", "").strip()

components = json.loads(clean_text)["components"]

components


[{'name': 'Usuários SEI', 'type': 'user'},
 {'name': 'AWS Shield', 'type': 'external_system'},
 {'name': 'Amazon CloudFront', 'type': 'external_system'},
 {'name': 'AWS WAF', 'type': 'external_system'},
 {'name': 'AWS Cloud', 'type': 'external_system'},
 {'name': 'sa-east-1 (São Paulo)', 'type': 'external_system'},
 {'name': 'Virtual Private Cloud', 'type': 'server'},
 {'name': 'Availability Zone A', 'type': 'external_system'},
 {'name': 'Availability Zone B', 'type': 'external_system'},
 {'name': 'Availability Zone C', 'type': 'external_system'},
 {'name': 'Public Subnet', 'type': 'server'},
 {'name': 'Private Subnet', 'type': 'server'},
 {'name': 'Application Load Balancer', 'type': 'server'},
 {'name': 'SEI / SIP (Auto Scaling API Server)', 'type': 'api'},
 {'name': 'Amazon Elastic File System (NFS) Multi-AZ', 'type': 'server'},
 {'name': 'Amazon RDS (Primary)', 'type': 'database'},
 {'name': 'Amazon RDS (Secondary)', 'type': 'database'},
 {'name': 'Amazon ElastiCache (memcached) Mu

In [ ]:
stride_map = {
    "server": ["Spoofing", "Tampering", "Denial of Service"],
    "database": ["Tampering", "Information Disclosure"],
    "api": ["Spoofing", "Repudiation"],
    "user": ["Spoofing"],
    "external_system": ["Spoofing", "Tampering"]
}

def analyze_stride(components):
    results = []

    for comp in components:
        threats = stride_map.get(comp["type"])
        if threats is None:
            threats = ["Unknown – No STRIDE mapping defined"]

        results.append({
            "component": comp["name"],
            "type": comp["type"],
            "threats": threats
        })

    return results

stride_results = analyze_stride(components)

stride_results


[{'component': 'Usuários SEI', 'type': 'user', 'threats': ['Spoofing']},
 {'component': 'AWS Shield',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'Amazon CloudFront',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'AWS WAF',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'AWS Cloud',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'sa-east-1 (São Paulo)',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'Virtual Private Cloud',
  'type': 'server',
  'threats': ['Spoofing', 'Tampering', 'Denial of Service']},
 {'component': 'Availability Zone A',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'Availability Zone B',
  'type': 'external_system',
  'threats': ['Spoofing', 'Tampering']},
 {'component': 'Availability Zone C',
  'type': 'external_system',
  'threats': 

In [ ]:
enrichment_prompt = f"""
Considere os resultados de STRIDE abaixo:

{json.dumps(stride_results, indent=2)}

Para cada ameaça, explique o racional técnico e sugira contramedidas.

Resposta em texto estruturado.
"""

response3 = client.responses.create(
    model="gpt-4.1-mini",
    input=enrichment_prompt
)

print(response3.output_text)


Abaixo segue a análise detalhada de cada ameaça identificada no modelo STRIDE aplicado ao sistema descrito, considerando os componentes e tipos de ameaças, com explicação técnica do racional e sugestões de contramedidas.

---

### 1. Spoofing (Falsificação de identidade)

**Racional técnico:**  
Spoofing refere-se à tentativa de um ator malicioso se passar por outro usuário, serviço ou componente legítimo para obter acesso não autorizado ou manipular dados e operações. É comum em autenticação insuficiente, intercâmbio inseguro de credenciais ou vulnerabilidades na validação de identidade.

**Componentes afetados:**  
Usuários SEI, AWS Shield, Amazon CloudFront, AWS WAF, AWS Cloud, sa-east-1 (São Paulo), Virtual Private Cloud, Availability Zones, Public/Private Subnet, Application Load Balancer, SEI / SIP API Server, Amazon Elastic File System, Solr Auto Scaling, AWS CloudTrail, AWS Key Management Service, AWS Backup, Amazon CloudWatch, Amazon SES.

**Contramedidas sugeridas:**

- Imple